# Problem definition

From description:

"The dataset is used for this competition is synthetic, but based on a real dataset and generated using a CTGAN. The original dataset deals with predicting the category on an eCommerce product given various attributes about the listing. Although the features are anonymized, they have properties relating to real-world features."

# Blend

Firstly the optimization of the blend weights with my submissions will be done using the `optim` (R base) function using the Nelder-Mead method

This blend will then be combined with external subsmissions using (the amazing) functions shared by [@mehrankazeminia](https://www.kaggle.com/mehrankazeminia) and [@somayyehgholami](https://www.kaggle.com/somayyehgholami) (thanks so much for sharing!)

<br>

<p align="right"><span style="color:firebrick">Dont forget the upvote if you liked the notebook! <i class="fas fa-hand-peace"></i></span> </p>

# Load dependencies

In [ ]:
import numpy as np 
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-jun-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-jun-2021/test.csv')
sub = pd.read_csv('../input/tabular-playground-series-jun-2021/sample_submission.csv')

In [ ]:
# load oof
oof1 = pd.read_csv("../input/tps-jun2021-r-nn-residual-keras/oof_nnresidual.csv", index_col='id')
oof2 = pd.read_csv("../input/tps-jun2021-r-wide-deep-keras/oof_widedeep.csv", index_col='id')
oof3 = pd.read_csv("../input/tps-jun2021-catboost-optuna/oof_cat_default.csv", index_col='id')
oof4 = pd.read_csv("../input/tps-jun2021-lightautoml/oof_lightautoml.csv", index_col='id')
oof5 = pd.read_csv("../input/tps-jun2021-lightgbm-optuna/oof_lgb_optuned.csv", index_col='id')
oof6 = pd.read_csv("../input/tps-jun2021-tabnetclassifier/oof_tab_optuned.csv", index_col='id')
oof7 = pd.read_csv("../input/tps-jun2021-r-nn-gbt-blend-optim-keras/oof_nn_gbt_blend.csv", index_col='id')
oof8 = pd.read_csv("../input/tps-jun2021-r-wide-deep-residual-optblend-keras/oof_widedeep_res.csv", index_col='id')

# load submission
sub1 = pd.read_csv("../input/tps-jun2021-r-nn-residual-keras/sub_nnresidual.csv", index_col='id')
sub2 = pd.read_csv("../input/tps-jun2021-r-wide-deep-keras/sub_widedeep.csv", index_col='id')
sub3 = pd.read_csv("../input/tps-jun2021-catboost-optuna/sub_cat_default.csv", index_col='id')
sub4 = pd.read_csv("../input/tps-jun2021-lightautoml/sub_lightautoml.csv", index_col='id')
sub5 = pd.read_csv("../input/tps-jun2021-lightgbm-optuna/sub_lgb_optuned.csv", index_col='id')
sub6 = pd.read_csv("../input/tps-jun2021-tabnetclassifier/sub_tab_default.csv", index_col='id')
sub7 = pd.read_csv("../input/tps-jun2021-r-nn-gbt-blend-optim-keras/sub_nn_gbt_blend.csv", index_col='id')
sub8 = pd.read_csv("../input/tps-jun2021-r-wide-deep-residual-optblend-keras/sub_widedeep_res.csv", index_col='id')

# External subs
sub1_ext = pd.read_csv("../input/3-tps-jun-21-smart-ensembling-for-classifier/submission_ens.csv", index_col='id')
sub2_ext = pd.read_csv("../input/tps-6-21-new-ensemble/blend_ens.csv", index_col='id')
sub3_ext = pd.read_csv("../input/tps06-nns-gbts-optimization/submission.csv", index_col='id')

# Blend optimization with Nelder-Mead

In [ ]:
# made in R:

#log_weight_loss <- function(w){
#    weighted_pred = as.matrix((oof1*w[1]+oof2*w[2]+oof3*w[3]+oof7*w[4]) / sum(w))
#    
#  r = yardstick::mn_log_loss_vec(
#    truth = factor(train$target),
#    estimate = weighted_pred)
#    
#    return(r)
#}

# results <- optim(rep(10, 4), log_weight_loss, method = "Nelder-Mead")
# w <- results$par

In [ ]:
w = [11.9028565343769, 6.57314783814548, 8.19327453286444, 12.3950366481336]

In [ ]:
oof_my_blend = ((oof1*w[0])+(oof2*w[1])+(oof3*w[2])+(oof7*w[3])) / np.sum(w)
sub_my_blend = ((sub1*w[0])+(sub2*w[1])+(sub3*w[2])+(sub7*w[3])) / np.sum(w)

# Smart Ensembling for Classifier

With external submissions. Thank you very much for sharing:

- https://www.kaggle.com/mehrankazeminia/3-tps-jun-21-smart-ensembling-for-classifier by [@mehrankazeminia](https://www.kaggle.com/mehrankazeminia) and [@somayyehgholami](https://www.kaggle.com/somayyehgholami); 
- https://www.kaggle.com/tunguz/tps-6-21-new-ensemble by [@tunguz](https://www.kaggle.com/tunguz);
- https://www.kaggle.com/hiro5299834/tps06-nns-gbts-optimization by [@hiro5299834](https://www.kaggle.com/hiro5299834)


see: https://www.kaggle.com/c/tabular-playground-series-jun-2021/discussion/248454

In [ ]:
def generation(main, support, coeff): 
    sub1  = support.copy()
    sub1v = sub1.values    
    
    sub2  = main.copy() 
    sub2v = sub2.values
       
    imp  = main.copy()    
    impv = imp.values  
    NCLASS = 9
    number = 0
    
    for i in range (len(main)):               
        
        row1 = sub1v[i,:]
        row2 = sub2v[i,:]
        row1_sort = np.sort(row1)
        row2_sort = np.sort(row2) 
        
        row = (row2 * coeff) + (row1 * (1.0 - coeff))
        row_sort = np.sort(row)        
        
        for j in range (NCLASS):             
            if ((row2[j] == row2_sort[8]) and (row1[j] != row1_sort[8])):                                
                row = row2 
                number = number + 1            
#               print(number, i)
        
        impv[i, :] = row
    
    imp.iloc[:, :] = impv[:, :]
    print(f'Number of rows unchanged: {number}')
    print(30 * '=')                                        
    return imp

In [ ]:
sub_gen = generation(sub3_ext, sub_my_blend, 0.85)
sub_gen = generation(sub2_ext, sub_gen, 0.85)
sub_gen = generation(sub1_ext, sub_gen, 0.85)

# Comparative Method for Classifier

see: https://www.kaggle.com/mehrankazeminia/2-tps-jun-21-comparative-method-for-classifier (many thanks for sharing)

In [ ]:
def improve(sub1, sub2, sub3, sub4, sub_ens, majority, m_majority):  
    
    sub1v = sub1.values
    sub2v = sub2.values
    sub3v = sub3.values
    sub4v = sub4.values
    
    imp = sub_ens.copy()
    impv = imp.values
    NCLASS = 9
    number = 0

    for i in range (len(sub_ens)):
        c_count = 0  
        row = impv[i,:]
        row_sort = np.sort(row)        
        
        row1 = sub1v[i,:]
        row2 = sub2v[i,:]
        row3 = sub3v[i,:]
        row4 = sub4v[i,:]
        row1_sort = np.sort(row1)
        row2_sort = np.sort(row2)
        row3_sort = np.sort(row3)
        row4_sort = np.sort(row4)
                      
        for j in range (NCLASS): 
            count = 0
            
            for k in range (NCLASS):                
                if (row4[j] == row4_sort[k]): 
                    
                    if (row1[j] == row1_sort[k]):
                        count = count + 1
                    if (row2[j] == row2_sort[k]):
                        count = count + 1                   
                    if (row3[j] == row3_sort[k]):
                        count = count + 1   
                        
            if (count >= majority):
                c_count = c_count + 1
        
        if ((c_count >= m_majority) and (row4_sort[8] >= row_sort[8])): 
            impv[i, :] = row4            
            number = number + 1            
#           print (number, i, c_count)                         
                    
    imp.iloc[:, 1:] = impv[:, 1:]
    p_number = round(((number / 100000) * 100),2)
    print('>>>  R  E  T  U  R  N  S  <<<')
    print(30 * '=')
    print(f'Number of changes: {number}\n')
    print(f'Percentage of changes: {p_number} %')
    print(30 * '=')
    return imp  

In [ ]:
sub_imp = improve(sub_my_blend, sub3_ext, sub2_ext, sub1_ext, sub_gen, 3, 5)

# Sub

In [ ]:
sub.iloc[:, 1:] = sub_imp.values
sub.to_csv("sub_blend_imp.csv", index=False)

<p align="right"><span style="color:firebrick">If you liked it, please don't forget to vote for the mentioned notebooks too! 🚀 </p>
